# Cartographie des Projets France2030 : Une Analyse Géographique des Financements ADEME

L'analyse présentée dans ce notebook s'appuie sur un jeu de données provenant de l'ADEME (Agence de l'environnement et de la maîtrise de l'énergie) concernant les projets financés dans le cadre de France2030. Le fichier source, disponible à l'adresse https://data.ademe.fr/data-fair/api/v1/datasets/zb29czhpiv4na0do47l6lmmg/lines?size=10000&page=1&format=csv, contient des informations détaillées sur les projets, notamment leur état d'avancement, les montants financiers impliqués, ainsi que les coordonnées géographiques des lieux principaux.

L'objectif principal de cette analyse est de préparer les données pour une visualisation cartographique qui met en évidence la répartition géographique des projets à travers la France. Plus précisément, il s'agit de créer une carte avec un choropleth des départements en fond de plan, en calculant le nombre de projets par département et en incluant les coordonnées géographiques des projets individuels.

## Méthodologie

La méthodologie employée pour cette analyse repose sur l'utilisation de DuckDB pour le traitement des données et de Plotly Express pour la visualisation. Tout d'abord, les données sont chargées et traitées à l'aide d'une requête SQL qui calcule le nombre de projets par département et extrait les coordonnées géographiques des projets. Ensuite, les résultats sont utilisés pour créer une carte choroplèthe des départements avec des marqueurs représentant les projets individuels.


Cette approche permet de combiner efficacement l'analyse de données et la visualisation pour offrir une représentation claire et informative de la répartition géographique des projets financés par l'ADEME dans le cadre de France2030.

 

Le contenu de ce notebook est disponible sous forme de sections détaillées qui exposent chaque étape de l'analyse, de la préparation des données à la visualisation finale. Des explications détaillées sont fournies pour chaque étape, ce qui facilite la compréhension et la reproduction de l'analyse.

## 🔧 Configuration

In [1]:
# Installation et imports
import duckdb as ddb
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

## 🦆 Chargement du dataset avec Duckdb

In [1]:
# Fonction de chargement complète (basée sur load_file_from_url_lite)
def load_file_from_url_lite(url_dataset="", loader="read_csv_auto", options="", nom_table="loaded_dataset", safe_mode=False):
    ddb.execute("install spatial")
    ddb.execute("load spatial")
    ddb.execute("INSTALL h3 FROM community")
    ddb.execute("LOAD h3")
    ddb.execute("install webbed from community;")
    ddb.execute("load webbed")
    ddb.execute("set force_download=True")
    ddb.execute(f"drop table if exists {nom_table}")   
    
    # Détection automatique du type de fichier
    if 'csv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'tsv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'txt' in url_dataset: 
        loader = "read_csv_auto"
    elif 'parquet' in url_dataset: 
        loader = "read_parquet"
    elif 'json' in url_dataset: 
        loader = "read_json_auto"
    elif 'xls' in url_dataset or 'xlsx' in url_dataset: 
        loader = "st_read"
    elif 'shp' in url_dataset: 
        loader = "st_read"
    elif 'geojson' in url_dataset: 
        loader = "st_read"
    elif 'xml' in url_dataset: 
        loader = "read_xml"
    elif 'html' in url_dataset: 
        loader = "read_html"
    else: 
        raise ValueError(f"Type de fichier non supporté pour {url_dataset}")
    
    if options=="": 
        options = "" 
    if 'csv' in url_dataset and safe_mode==True: 
        options = ", all_varchar=1" 
    if nom_table=="": 
        nom_table = "loaded_dataset"
    
    try:
        status = ddb.sql(f"""
            create or replace table {nom_table} as select *
            from
            {loader}("{url_dataset}" {options})
        """)
        return status
    except Exception as e:
        return f"Erreur au chargement du fichier : {str(e)}"

def run_query(sql):
    return ddb.sql(sql.replace("`"," ")).to_df()

# Chargement des données
load_file_from_url_lite("https://data.ademe.fr/data-fair/api/v1/datasets/zb29czhpiv4na0do47l6lmmg/lines?size=10000&page=1&format=csv", safe_mode=True)
print("✅ Données chargées avec succès")

## 🔍 Analyse SQL

Cette requête utilise des techniques SQL pour extraire et transformer les données de manière efficace.

In [2]:
# Exécution de la requête
df = run_query(""" WITH departement_projects AS (
 SELECT 
 SUBSTRING("Lieu principal département code",1,2) AS code_dept,
 COUNT(*) AS nb_projets_dept
 FROM 
 loaded_dataset
 GROUP BY 
 SUBSTRING("Lieu principal département code",1,2)
),
project_data AS (
 SELECT 
 SUBSTRING("Lieu principal département code",1,2) AS code_dept,
 "Nom du projet" AS description_projet,
 CAST("Lieu principal commune longitude (X)" AS DOUBLE) AS x_projet,
 CAST("Lieu principal commune latitude (Y)" AS DOUBLE) AS y_projet
 FROM 
 loaded_dataset
)
SELECT 
 pd.code_dept,
 dp.nb_projets_dept,
 pd.description_projet,
 pd.x_projet,
 pd.y_projet
FROM 
 project_data pd
LEFT JOIN departement_projects dp ON pd.code_dept = dp.code_dept
ORDER BY 
 pd.code_dept """)
print(f"Résultats : {len(df)} lignes")
df.head()

## 📈 Visualisation

La bibliothèque principale utilisée est Plotly Express, qui permet de créer des visualisations interactives et attractives. Cette technologie est adaptée pour représenter des données géographiques, comme ici une carte choroplèthe des départements français et des marqueurs de projets, grâce à ses fonctionnalités de cartographie avancées. Cela permet une exploration intuitive des données.

In [3]:
import pandas as pd
import duckdb as ddb
import pandas as pd
import plotly.express as px
import json
import requests

# Téléchargement du GeoJSON des départements
url = 'https://france-geojson.gregoiredavid.fr/repo/departements.geojson'
resp = requests.get(url)
geojson_dept = resp.json()

# DataFrame département / nombre de projets uniques
dept_proj = df[['code_dept','nb_projets_dept']].drop_duplicates('code_dept')

# Carte choroplèthe des départements
fig = px.choropleth_mapbox(
    dept_proj,
    geojson=geojson_dept,
    locations='code_dept',
    featureidkey='properties.code',
    color='nb_projets_dept',
    color_continuous_scale='YlGnBu',
    mapbox_style='carto-positron',
    zoom=4.3,
    center={'lat':46.5,'lon':2.5},
    opacity=0.65,
    labels={'nb_projets_dept':'Nb projets<br>ADEME'}
)

# Filtre lat/lon pour rester en métropole
mask = (
    (df['y_projet'] >= 41) & (df['y_projet'] <= 52) &
    (df['x_projet'] >= -6) & (df['x_projet'] <= 10)
)
df_ = df[mask].copy()

# Marqueurs des projets
scatter = px.scatter_mapbox(
    df_,
    lat='y_projet',
    lon='x_projet',
    hover_name='description_projet',
    hover_data={'y_projet':False,'x_projet':False},
    opacity=0.9,
    color_discrete_sequence=['#ff4b4b']
)
for trace in scatter.data:
    fig.add_trace(trace)

fig.update_layout(
    title={
        'text':'France2030 – Projets financés par l’ADEME',
        'x':0.5,
        'xanchor':'center',
        'font':{'size':18,'family':'Roboto'}
    },
    margin={'l':20,'r':20,'t':50,'b':20},
    height=650
)

dataviz = fig
dataviz

---
*Made with ❤️ and with [duckit.fr](https://duckit.fr) - [Ali Hmaou](https://www.linkedin.com/in/ali-hmaou-6b7b73146/)*